In [1]:
import pandas as pd
import sys
import os
import re

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [2]:
module_path = os.path.abspath(os.path.join('..'))    

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from src.db.session import engine

query = "SELECT * FROM ads"

df = pd.read_sql(query, con=engine)

In [4]:
display(df.head())

,avito_id,url,title,price,description,location,published_at,condition,seller_name,seller_rating,seller_reviews_count
0,7695007829,https://www.avito.ru/moskva/telefony/iphone_11...,"iPhone 11, 64 ГБ",10000,"Продаю айфон 11, айклауд взломали.",Москва,2025-10-08 16:01:32.990590+00:00,Б/у,Анна,4.8,16.0
1,7743661402,https://www.avito.ru/moskva/telefony/iphone_11...,"iPhone 11, 64 ГБ",10998,"Продаю iphone 11 на 64гб в хорошем состоянии, ...",Москва,2025-10-08 14:01:32.998022+00:00,Б/у,None,NaN,NaN
2,7684049927,https://www.avito.ru/moskva/telefony/iphone_11...,"iPhone 11, 64 ГБ",9000,Телефон в рабочем состоянии. Трещина не мешают...,Москва,2025-10-08 13:01:33.003023+00:00,Б/у,None,NaN,NaN
3,7789880055,https://www.avito.ru/moskva/telefony/iphone_11...,"iPhone 11, 64 ГБ",9500,"Телефон в хорошем состоянии, работает отлично....",Москва,2025-10-08 13:01:33.007280+00:00,Б/у,None,NaN,NaN
4,7674616969,https://www.avito.ru/moskva/telefony/iphone_11...,"iPhone 11, 64 ГБ",12990,Телефон в отличном состоянии!\n\nРодной Акб 89...,Москва,2025-10-08 13:01:33.011891+00:00,Б/у,AppleRoom,5.0,2.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4451 entries, 0 to 4450
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   avito_id              4451 non-null   int64              
 1   url                   4451 non-null   object             
 2   title                 4451 non-null   object             
 3   price                 4451 non-null   int64              
 4   description           4450 non-null   object             
 5   location              4451 non-null   object             
 6   published_at          4451 non-null   datetime64[ns, UTC]
 7   condition             4451 non-null   object             
 8   seller_name           1778 non-null   object             
 9   seller_rating         1778 non-null   float64            
 10  seller_reviews_count  1778 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(2), object(6)
memory usa